In [3]:
import pandas as pd
import plotly.express as px
import numpy as np
import json
import matplotlib.pyplot as plt
import plotly.io as pio
import pymongo
pio.renderers.default = 'browser'
errors=np.seterr(divide = 'ignore') 
%matplotlib inline

In [4]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient.Jcomp
states_to_drop=['Unassigned','Dadra and Nagar Haveli and Daman and Diu','Cases being reassigned to states','Telangana','Ladakh']
Dates_to_use=["2020-04-01","2020-05-01","2020-06-01","2020-07-01","2020-08-01","2020-09-01","2020-10-01",
              "2020-11-01","2020-12-01","2021-01-01","2021-02-01","2021-03-01","2021-04-01","2021-05-01"]
query1={"$match":{"$and":[{"State/UnionTerritory":{"$nin":states_to_drop}},{"Date":{"$in":Dates_to_use}}]}}
query2={"$project":{"_id":0,"Date":1,"State/UnionTerritory":1,"Cured":1,"Deaths":1,"Confirmed":1}}
df=pd.DataFrame(list(db.covid_acc.aggregate([query1,query2])))
df["Confirmed_log"]=np.log10(df["Confirmed"])
df["Cured_log"]=np.log10(df["Cured"])
df["Deaths_log"]=np.log10(df["Deaths"])

In [5]:
india_states = json.load(open("states_india.geojson",'r'))
state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]
df["id"] = df["State/UnionTerritory"].apply(lambda x: state_id_map[x])

In [6]:
fig= px.choropleth(
    df,
    locations="id",
    geojson=india_states,
    color="Confirmed_log",
    hover_name="State/UnionTerritory",
    hover_data=["Confirmed"],
    title="India Confirmed cases map",
    fitbounds="locations",
    animation_frame='Date'
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [139]:
fig = px.choropleth(
    df,
    locations="id",
    geojson=india_states,
    color="Cured_log",
    hover_name="State/UnionTerritory",
    hover_data=["Cured"],
    title="India Cured cases map",
    fitbounds="locations",
    animation_frame='Date'
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [140]:
fig = px.choropleth(
    df,
    locations="id",
    geojson=india_states,
    color="Deaths_log",
    hover_name="State/UnionTerritory",
    hover_data=["Deaths"],
    title="India Death cases map",
    fitbounds="locations",
    animation_frame='Date'
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()